## Shopify Data Science Internship Challenge

### Question 1: 
*On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13.*

*Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis.*


-----


**a. Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.** 

-----



I converted the csv file to a dataframe to look through its contents and find out why the average result was skewed.
Looking through the description of the dataset, I could see there was a maximum value of 704,000. This and other outlier high values could be skewing the standard deviation, causing the average to be off.

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv")
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [2]:
print(df.isnull().sum().sum()!=0) #Checking for null values
df.describe() 

False


,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


From the description above, we can see the misleading average under order_amount, \$3,145.13.

The standard deviation is 41,282, a surprisingly high amount for shoe sales.

We can check the minimum value for order_amount, which is \$90, and the maximum value, which is \\$704,000. It's likely that our average is being skewed by a few very high order amounts, which would explain the high standard deviation as well.

In [3]:
# What other high values are there?
np.sort(df['order_amount'].unique())[-10:]

array([  1064,   1086,   1408,   1760,  25725,  51450,  77175, 102900,
       154350, 704000])

In [4]:
# How many times was a value over 25000 present?
total = df['order_amount'].astype(int) > 25000
total.sum()

63

There are 63 orders valued at over 25,000, skewing the average.

-----

**b. What metric would you report for this dataset?**

-----


1. We could use the median, which would give a more 'middle' value.

2. We could also use a trimmed mean, which removes outliers. 

3. Another option is to get the average for all stores individually.

4. Or, we can separate the dataset into wholesale and individual based on number of items sold


-----

**c. What is its value?**

-----


1. Using median:

In [5]:
df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


The 50% distribution under order_amount gives us a middle value. Using this, we get the median value \$284.

In [6]:
# Is the median value of $284 correct?
df['order_amount'].median()==284

True

2. Using trimmed mean:

In [7]:
from scipy import stats
tm = stats.trim_mean(df['order_amount'], 0.1) #Trims 10%

print("The trimmed mean is: ", round(tm, 2))

The trimmed mean is:  287.39


3. Calculating the average for each store:

In [8]:
df.groupby('shop_id')['order_amount'].mean()[40:80]

shop_id
41       254.000000
42    235101.490196
43       333.913793
44       262.153846
45       269.310345
46       347.441860
47       259.148936
48       242.775000
49       279.905660
50       403.545455
51       361.804348
52       316.926829
53       214.117647
54       276.640000
55       327.750000
56       218.189189
57       296.773585
58       254.949153
59       358.966667
60       350.234043
61       344.440000
62       308.837209
63       264.965517
64       272.186047
65       330.814815
66       312.886792
67       272.621622
68       254.638298
69       264.183333
70       343.067797
71       323.030303
72       309.565217
73       335.689655
74       306.000000
75       240.761905
76       321.071429
77       280.800000
78     49213.043478
79       328.481481
80       299.666667
Name: order_amount, dtype: float64

We can see from the output that most of the outliers come from 2 stores, shop42 & shop78.

We can remove these stores from our calculations:

In [9]:
avg = df[~df['shop_id'].isin(['42', '78'])].order_amount.mean()
print("The average without the outlier stores is: $", round(avg, 2))

The average without the outlier stores is: $ 300.16


4. Separating data into wholesale & individual:

In [10]:
wholesale = df[df['total_items'].astype(int) >= 5]
individual = df[df['total_items'].astype(int) < 5]

In [11]:
print("The wholesale average (Sales 5 items or over) is $", round(wholesale['order_amount'].mean(), 2))
print("The average for an individual order (Sale under 5 items) is $", round(individual['order_amount'].mean(), 2))

The wholesale average (Sales 5 items or over) is $ 117201.87
The average for an individual order (Sale under 5 items) is $ 722.35


This average is still higher than the other metrics used, but much lower than the 3,145 initially calculated.